In [ ]:
"""
1. 創造欄位: "標題和來源網站"、"網站"、"頻道"、"月"、"日"、"時"、"日期"、"日月年"
 - 註: 連續變項: 聲量、好感度; 類別變項: 標題、頻道、時間 (主題、族群)
2. 計算P/N值 (根據自定義單位)
 - 註: 需要計算回應比
3. 查看文章內容 (根據標題)
4. 時間段內文章聲量排行
5. 頻道聲量排行和文章聲量排行 (可用分類聲量排行取代)
6. 分類聲量排行 (可自訂聲量單位，根據欄位Label)
 - 註: 把分類和根據聲量單位分開寫
7. 擴散: 時間頻率切段 (重點)，根據自訂單位排行聲量
8. 聲量趨勢圖、正負聲量+PN值趨勢圖
"""

In [2]:
import pandas as pd
from tqdm import trange

In [3]:
path = r"C:\Users\kaohunglin\Desktop\報告\台東年度議題\data\完整資料"
file_name = "Result_台東熱氣球嘉年華"
df = pd.read_csv(f"{path}\{file_name}.csv", encoding='utf_8_sig')
df.drop(columns=["Unnamed: 0"], inplace=True)
# df.head()

# 合併"標題和來源網站"及分割"網站"與"頻道"

In [4]:
# 創造"標題和來源網站"欄位
df["標題和來源網站"] = df.apply(lambda x: str(x.標題)+str(x.來源網站), axis=1)

In [5]:
df["網站"] = df.來源網站.apply(lambda x: x.split(" > ")[0])
df["頻道"] = df.來源網站.apply(lambda x: x.split(" > ")[1])

# 時間標記欄位

In [6]:
# df["發佈時間_月"] = df.發佈時間.apply(lambda x: x.split(" ")[0].split("/")[1])
# df["發佈時間_日"] = df.發佈時間.apply(lambda x: x.split(" ")[0].split("/")[2])
# df["發佈時間_時"] = df.發佈時間.apply(lambda x: x.split(" ")[1].split(":")[0])
# df[["發佈時間","發佈時間_月","發佈時間_日","發佈時間_時"]].head()

,發佈時間,發佈時間_月,發佈時間_日,發佈時間_時
0,2021/10/31 14:12:52,10,31,14
1,2021/10/31 12:46:54,10,31,12
2,2021/10/30 17:53:35,10,30,17
3,2021/10/30 11:24:14,10,30,11
4,2021/10/30 03:52:00,10,30,03


In [7]:
# df["發佈時間_日期"] = df.apply(lambda x: str(x.發佈時間_月)+"/"+str(x.發佈時間_日), axis=1)
# df["發佈時間_年月日"] = df.發佈時間.apply(lambda x: x.split(" ")[0])

In [9]:
def create_channel_and_time_tags(df):
    df["標題和來源網站"] = df.apply(lambda x: str(x.標題)+str(x.來源網站), axis=1)
    df["網站"] = df.來源網站.apply(lambda x: x.split(" > ")[0])
    df["頻道"] = df.來源網站.apply(lambda x: x.split(" > ")[1])
    df["發佈時間_月"] = df.發佈時間.apply(lambda x: x.split(" ")[0].split("/")[1])
    df["發佈時間_日"] = df.發佈時間.apply(lambda x: x.split(" ")[0].split("/")[2])
    df["發佈時間_時"] = df.發佈時間.apply(lambda x: x.split(" ")[1].split(":")[0])
    df["發佈時間_日期"] = df.apply(lambda x: str(x.發佈時間_月)+"/"+str(x.發佈時間_日), axis=1)
    df["發佈時間_年月日"] = df.發佈時間.apply(lambda x: x.split(" ")[0])

In [10]:
create_channel_and_time_tags(df)

# 計算P/N值

In [ ]:
def cal_PN_ratio(group_by_cat):
    # 計算總評論數
    df_total_count = df.groupby([group_by_cat]).size().to_frame().reset_index()
    df_total_count.rename(columns={0:"總評論數"}, inplace=True)
    # 計算正面評論數
    df_positive_count = df[df.情緒標記=="正面"].groupby([group_by_cat]).size().to_frame().reset_index()
    df_positive_count.rename(columns={0:"正面評論數"}, inplace=True)
    # 計算負面評論數
    df_negative_count = df[df.情緒標記=="負面"].groupby([group_by_cat]).size().to_frame().reset_index()
    df_negative_count.rename(columns={0:"負面評論數"}, inplace=True)
    # 合併dataframes
    df_sentiment = (df_positive_count.merge(df_negative_count, on=[group_by_cat], how="outer")
                                     .merge(df_total_count, on=[group_by_cat], how="outer")
                                     .fillna(0))
    # 計算情緒正負比 (P+1/N+1)
    df_sentiment["P/N比"] = df_sentiment.apply(lambda x: (x.正面評論數+1)/(x.負面評論數+1), axis=1)
    return df_sentiment

In [ ]:
group_by_cat = "發佈時間_日期"
df_sentiment = cal_PN_ratio(group_by_cat)

# 查看文章內容

In [18]:
def content_of_article(article,label,start,end,column_name=""):
    print("主文內容:")
    print(df[(df[label]==article)&(df["主文/回文"]=="主文")].內容)
    print("\n回文內容:")
    print(df[(df[label]==article)&(df["主文/回文"]!="主文")].內容.iloc[start:end])
    if len(column_name) > 0:
        print(f"\n{column_name}:")
        print(df[(df[label]==article)&(df["主文/回文"]!="主文")][column_name].unique())

#  期間聲量排行

In [15]:
df["id"] = [x for x in range(len(df))]

In [16]:
def period_rank(start_date,end_date):
    start = df[df["發佈時間_日期"]==end_date].id.iloc[0]
    end = df[df["發佈時間_日期"]==start_date].id.iloc[-1]
    df_period_rank = df[start:end+1].copy()
    df_period_rank = df_period_rank.groupby(["標題和來源網站"]).size().to_frame().rename(columns={0:"聲量"}).sort_values(by=["聲量"], ascending=False)
    return df_period_rank

In [ ]:
period_rank("07/23","07/23")

In [ ]:
def period_rank_ymd(start_date,end_date,group_cat="標題和來源網站"):
    start = df[df["發佈時間_年月日"]==end_date].id.iloc[0]
    end = df[df["發佈時間_年月日"]==start_date].id.iloc[-1]
    df_period_rank = df[start:end+1].copy()
    df_period_rank = df_period_rank.groupby([group_cat]).size().to_frame().rename(columns={0:"聲量"}).sort_values(by=["聲量"], ascending=False)
    return df_period_rank

# 分類聲量排行

## 頻道聲量排行

In [ ]:
def channel_rank(df, rank_to):
    df_channel_rank = df.groupby(["來源網站"]).size().to_frame().rename(columns={0:"聲量"}).sort_values(by=["聲量"],ascending=False)
    df_channel_rank = df_channel_rank[:rank_to]
    return df_channel_rank

## 文章聲量排行

In [ ]:
def article_ranking(df, rank_to):
    df_article_rank = df.groupby(["標題和來源網站"]).size().to_frame().rename(columns={0:"聲量"}).sort_values(by=["聲量"],ascending=False).reset_index()
    df_article_rank = df_article_rank[:rank_to]
    df_article_rank["標題"] = ""
    df_article_rank["頻道"] = ""
    df_article_rank["日期"] = ""
    for i in range(len(df_article_rank)):
        article = df_article_rank.標題和來源網站.iloc[i]
        df_article_rank.at[i,"標題"] = df[df.標題和來源網站 == article].標題.iloc[0].replace("?","")
        df_article_rank.at[i,"頻道"] = df[df.標題和來源網站 == article].來源網站.iloc[0]
        df_article_rank.at[i,"日期"] = df[df.標題和來源網站 == article].發佈時間_日期.iloc[0]
    df_article_rank.drop(columns=["標題和來源網站"], inplace=True)
    return df_article_rank

## 分類聲量排行

In [27]:
def create_cat_list(predefined_cat_list,category):
    if predefined_cat_list != "":
        cat_list = predefined_cat_list
    else:
        cat_list = df[category].unique()
    return cat_list

def volume_by_cat(category,cat_part,mode,display_num=5,path="",f_name="",input_cat_list=""):
    cat_list = create_cat_list(input_cat_list,category)
    cat_total_list = []
    for cat in cat_list: # 每一個cat in cat_list會做一個排序
        # 根據cat_part排序聲量
        df_cat = df[df[category]==cat].groupby([cat_part]).size().to_frame().rename(columns={0:"聲量"}).sort_values(by=["聲量"],ascending=False)
        # 每個cat的總聲量
        cat_total = len(df[df[category]==cat])
        cat_total_list.append(cat_total)
        if mode == "display":
            print(f"{cat}總聲量: {cat_total}")
            print(df_cat[:display_num],"\n")
        elif mode == "output":
            # 輸出每個cat的排序
            df_cat.to_excel(f"{path}/{f_name}_{cat}分類聲量排行.xlsx")
    if mode == "output":
        # 輸出cat_total_list
        df_cat_total_list = pd.DataFrame({category:cat_list,"聲量":cat_total_list})

In [ ]:
path = "C:/Users/kaohunglin/Desktop/報告/台東年度議題/工作表整理"
f_name = "台東年度議題_熱氣球嘉年華"
input_cat_list = list(df.發佈時間_日期.unique()[47:79])
volume_by_cat("發佈時間_日期","標題",path,f_name,input_cat_list)

In [28]:
volume_by_cat("來源","標題和來源網站",mode="display",display_num=10)

社群網站總聲量: 39102
                                                      聲量
標題和來源網站                                                 
號外號外，這顆大球難道是??臺北市首次以吉祥物熊讚Bravo，製作3D立體超...Facebo...  4211
#文末有賞夏季的台東少了熱氣球，就像水寶手上少了FIN那個畫面偶～不～敢～想...Facebo...  2377
【HELLO在哪裡??】 #文末有抽獎活動現在各地好朋友來到台東時??應該都...Facebo...  1822
??#按讚分享抽全聯商品禮券??第四集「聊瞭台東」有沒有讓大家認識更多的臺東美...Faceb...  1733
??你們睡醒了沒呀？就是今天！ 8/14今夏最萌的❤️臺北熊讚熱氣球即...Facebook粉...  1436
#東森購物網#台東熱氣球嘉年華#留言有驚喜台東縣舉辦熱氣球已經超過十年了...Facebook...  1097
??送你限量的熱氣球嘉年華口罩你要嗎??簡單哥來抽獎嘍???? ??參加方式：...Faceb...  1055
感情就像熱氣球的飛行員，有風的時候領你乘風翱翔，沒風的時候也能帶你平安落地。 ...Faceb...  1016
????????大會報告~大會報告?? ✨✨熱氣球嘉年華全新企劃登場✨✨前所未...Faceb...  1001
??熊讚熱氣球第二波的抽獎活動來啦！臺北熊讚熱氣球X希望啟航熱騰騰的影片...Facebook...   971 

新聞總聲量: 4611
                                                 聲量
標題和來源網站                                            
台東熱氣球飛到一半突著火10年來最嚴重飛安事故yahoo新聞 > 首頁-各家新聞        179
Delta病毒入侵＋強颱燦樹恐直撲台！台東熱氣球嘉年華提前閉幕LINE TODAY > 生活   52
首搭19米熊讚熱氣球亮相！柯文哲搔頭苦笑：在想會不會怕LINE TODAY > 生活       50
柯P嗆陳其邁「太出頭」高市府回酸：市長忙勘災沒空搭熱氣